In [ ]:
# 2025-11-17 run it from "python 3" (ipykernel) on gl-base-env-j4-ces with 1 L4 gpu


# preparing data

## copy data from gcs bucket 
merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data

In [ ]:
# gsutil ls -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data/
# gsutil -m cp -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data_new/polyhaven_tiny_13_new_pt /home/jupyter/mluser/git/cube/data/dataspace/

# (base) jupyter@gl-base-env-j4-ces:~/mluser/git/cube/data/dataspace/polyhaven_tiny$ ls -lh
# total 1.6G
# -rw-r--r-- 1 jupyter jupyter 1.5G Oct 22 00:03 cubediff_train.tar
# -rw-r--r-- 1 jupyter jupyter 111M Oct 22 00:02 cubediff_val.tar
# drwxr-xr-x 2 jupyter jupyter  36K Oct 22 00:02 latents
# drwxr-xr-x 6 jupyter jupyter 4.0K Oct 22 00:03 raw


# set up env

# install packages

In [ ]:
# !pip install transformers==4.35.2 \
#     diffusers==0.25.1 huggingface_hub==0.25.1 \
#     accelerate==0.22.0 -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


In [5]:
!pip install torch==2.1.2 torchvision==0.16.2 \
    opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
    xformers requests pillow openexr opencv-python-headless wandb \
    bitsandbytes webdataset timm imageio \
    diffusers==0.25.1 huggingface_hub==0.25.1 transformers==4.35.2 \
    accelerate==0.22.0 deepspeed \
    -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple

# !pip uninstall -y torch torchvision \
#     opencv-python opencv-contrib-python matplotlib tqdm einops \
#     xformers requests pillow openexr opencv-python-headless wandb \
#     bitsandbytes webdataset timm imageio \
#     diffusers huggingface_hub transformers \
#     accelerate deepspeed \
    

# !pip install torch==2.1.2 torchvision==0.16.2 \
#     opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
#     xformers requests pillow openexr opencv-python-headless wandb \
#     bitsandbytes webdataset timm imageio \
#     diffusers huggingface_hub transformers \
#     accelerate deepspeed \
#     -i https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


Looking in indexes: https://artifactory.tools.bestbuy.com/artifactory/api/pypi/python-virtual/simple


In [2]:
!ls -lh /opt/conda/lib/python3.10/site-packages/transformers/integrations/


total 152K
-rw-r--r-- 1 jupyter jupyter 4.3K Nov 18 01:43 __init__.py
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 01:43 __pycache__
-rw-r--r-- 1 jupyter jupyter 4.5K Nov 18 01:43 awq.py
-rw-r--r-- 1 jupyter jupyter  14K Nov 18 01:43 bitsandbytes.py
-rw-r--r-- 1 jupyter jupyter  18K Nov 18 01:43 deepspeed.py
-rw-r--r-- 1 jupyter jupyter  71K Nov 18 01:43 integration_utils.py
-rw-r--r-- 1 jupyter jupyter  22K Nov 18 01:43 peft.py


## check packages versions

In [4]:
!pip list | grep "diff\|hug\|acc\|trans\|torch\|peft\|tok\|xformers"

accelerate                               0.22.0
asttokens                                3.0.0
diffusers                                0.25.1
huggingface-hub                          0.25.1
sentence-transformers                    5.1.2
tokenizers                               0.15.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2
xformers                                 0.0.23.post1


In [13]:
import transformers, inspect
print("Transformers version:", transformers.__version__)
print("Loaded from:", inspect.getfile(transformers))


AttributeError: module 'torch.utils._pytree' has no attribute 'register_pytree_node'

In [15]:
import accelerate, inspect
print(accelerate.__version__)
print(inspect.getfile(accelerate))


1.11.0
/opt/conda/lib/python3.10/site-packages/accelerate/__init__.py


In [14]:
import OpenEXR

In [7]:
!pip show OpenEXR

Name: OpenEXR
Version: 3.4.2
Summary: Python bindings for the OpenEXR image file format
Home-page: https://openexr.com
Author: 
Author-email: Contributors to the OpenEXR project <info@openexr.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy
Required-by: 


# config shm

In [8]:
!df -h

Filesystem       Size  Used Avail Use% Mounted on
udev              32G     0   32G   0% /dev
tmpfs            6.3G  704K  6.3G   1% /run
/dev/nvme0n1p1   148G   45G   98G  32% /
tmpfs             32G   20K   32G   1% /dev/shm
tmpfs            5.0M     0  5.0M   0% /run/lock
/dev/nvme0n1p15  124M   11M  114M   9% /boot/efi
/dev/nvme0n2     147G   42G  106G  29% /home/jupyter


In [9]:
# allocate more memory for shm
!sudo mount -o remount,size=32G /dev/shm

In [10]:
!df -h

Filesystem       Size  Used Avail Use% Mounted on
udev              32G     0   32G   0% /dev
tmpfs            6.3G  704K  6.3G   1% /run
/dev/nvme0n1p1   148G   45G   98G  32% /
tmpfs             32G   20K   32G   1% /dev/shm
tmpfs            5.0M     0  5.0M   0% /run/lock
/dev/nvme0n1p15  124M   11M  114M   9% /boot/efi
/dev/nvme0n2     147G   42G  106G  29% /home/jupyter


# copied HF/hub and torch.hub from gcs to this VM

In [ ]:
# on target VM:
# 2025-11-17 this is important to copy the new "models--openai--clip-vit-base-patch32/"
# !gsutil -m cp -r gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub1/* \
#   ~/.cache/huggingface/hub/


In [ ]:
!ls -lh ~/.cache/huggingface/hub

In [ ]:
# 2025-11-17 no need to run
# on target VM:
# mkdir -p ~/.cache/torch/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/


In [11]:
!ls -lh ~/.cache/torch/hub/

total 4.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Oct 25 06:27 checkpoints


In [12]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 jupyter jupyter 528M Oct 25 06:27 vgg16-397923af.pth


# check models

In [13]:
!ls -lh ~/.cache/huggingface/hub/

total 24K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 17 19:40 clip-vit-base-patch32
drwxr-xr-x 5 jupyter jupyter 4.0K Nov 18 00:03 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 jupyter jupyter 4.0K Oct 25 06:19 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version.txt
-rw-r--r-- 1 jupyter jupyter    1 Nov 17 19:40 version_diffusers_cache.txt


In [14]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/:
total 12K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:03 blobs
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 07:01 refs
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 07:55 snapshots

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 jupy

In [15]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs 

/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs:
total 1.2G
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 182766ce89b439768edadda342519f33802f5364
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 4fdaf6842dd5a725b940c92b6b692490ce59d548
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 564c0ebd5ce29c4ee4864004aee693deadd3128c
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 5a12a1eb250987a4eee0e3e7d7338c4b22724be1
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 9bfb42aa97dcd61e89f279ccaee988bccb4fabae
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 a2a88b96561196777ca173b15309ea859f4d2ce0
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 a63082132ba4f97a80bea76823f544493bffa8082296d62d71581a4feff1576f
-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 bbfec752c9a675946c6dce106def6f35c882dcc2


In [16]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364

-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/182766ce89b439768edadda342519f33802f5364


In [17]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548

-rw-r--r-- 1 jupyter jupyter 592 Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/4fdaf6842dd5a725b940c92b6b692490ce59d548


In [18]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c

-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/564c0ebd5ce29c4ee4864004aee693deadd3128c


In [19]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/9bfb42aa97dcd61e89f279ccaee988bccb4fabae

-rw-r--r-- 1 jupyter jupyter 389 Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/9bfb42aa97dcd61e89f279ccaee988bccb4fabae


In [20]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/a2a88b96561196777ca173b15309ea859f4d2ce0

-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/a2a88b96561196777ca173b15309ea859f4d2ce0


In [21]:
!ls -lhR ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/bbfec752c9a675946c6dce106def6f35c882dcc2

-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 /home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/blobs/bbfec752c9a675946c6dce106def6f35c882dcc2


In [22]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/

total 4.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 00:03 3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268


In [23]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268

total 581M
-rw-r--r-- 1 jupyter jupyter 4.1K Nov 18 00:03 config.json
-rw-r--r-- 1 jupyter jupyter 513K Nov 18 00:03 merges.txt
-rw-r--r-- 1 jupyter jupyter  316 Nov 18 00:03 preprocessor_config.json
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 00:03 pytorch_model.bin
-rw-r--r-- 1 jupyter jupyter  389 Nov 18 00:03 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter 2.2M Nov 18 00:03 tokenizer.json
-rw-r--r-- 1 jupyter jupyter  592 Nov 18 00:03 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 00:03 vocab.json


In [24]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/182766ce89b439768edadda342519f33802f5364

ls: cannot access '/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/182766ce89b439768edadda342519f33802f5364': No such file or directory


In [25]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/refs/

total 8.0K
-rw-r--r-- 1 jupyter jupyter   40 Nov 18 00:03 main
drwxr-xr-x 3 jupyter jupyter 4.0K Nov 18 00:03 refs


In [26]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 jupyter jupyter 528M Oct 25 06:27 vgg16-397923af.pth


In [27]:
!ls -lh /opt/conda/lib/python3.10/site-packages/transformers-*

total 428K
-rw-r--r-- 1 jupyter jupyter    4 Nov 18 01:43 INSTALLER
-rw-r--r-- 1 jupyter jupyter  12K Nov 18 01:43 LICENSE
-rw-r--r-- 1 jupyter jupyter 121K Nov 18 01:43 METADATA
-rw-r--r-- 1 jupyter jupyter 273K Nov 18 01:43 RECORD
-rw-r--r-- 1 jupyter jupyter    0 Nov 18 01:43 REQUESTED
-rw-r--r-- 1 jupyter jupyter   92 Nov 18 01:43 WHEEL
-rw-r--r-- 1 jupyter jupyter   81 Nov 18 01:43 entry_points.txt
-rw-r--r-- 1 jupyter jupyter   13 Nov 18 01:43 top_level.txt


In [28]:
!ls -lh /opt/conda/lib/python3.10/site-packages/transformers/integrations

total 152K
-rw-r--r-- 1 jupyter jupyter 4.3K Nov 18 01:43 __init__.py
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 01:43 __pycache__
-rw-r--r-- 1 jupyter jupyter 4.5K Nov 18 01:43 awq.py
-rw-r--r-- 1 jupyter jupyter  14K Nov 18 01:43 bitsandbytes.py
-rw-r--r-- 1 jupyter jupyter  18K Nov 18 01:43 deepspeed.py
-rw-r--r-- 1 jupyter jupyter  71K Nov 18 01:43 integration_utils.py
-rw-r--r-- 1 jupyter jupyter  22K Nov 18 01:43 peft.py


In [29]:
!ls -lh /opt/conda/lib/python3.10/site-packages/accelerate-*

total 60K
-rw-r--r-- 1 jupyter jupyter   4 Nov 18 01:43 INSTALLER
-rw-r--r-- 1 jupyter jupyter 12K Nov 18 01:43 LICENSE
-rw-r--r-- 1 jupyter jupyter 18K Nov 18 01:43 METADATA
-rw-r--r-- 1 jupyter jupyter 12K Nov 18 01:43 RECORD
-rw-r--r-- 1 jupyter jupyter   0 Nov 18 01:43 REQUESTED
-rw-r--r-- 1 jupyter jupyter  92 Nov 18 01:43 WHEEL
-rw-r--r-- 1 jupyter jupyter 175 Nov 18 01:43 entry_points.txt
-rw-r--r-- 1 jupyter jupyter  11 Nov 18 01:43 top_level.txt


## test they can be loaded from local cache

In [30]:
from transformers import CLIPModel, CLIPProcessor

# This should now work without network calls
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True, local_files_only=False) # use_safetensors=True, caused the error of no safetensors for the downloaded model.
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=False, local_files_only=True) # worked for the model downloaded on 2025-10-29 in "bbyus-trn-amlo-yieldoptimz-p01"
                                                                                                                # details at mluser/git/cube/training/notebooks/v2/download_copy_vit.ipynb
                                                                                                                # but the same vit model could be loaded well with "use_safetensors=True" 
                                                                                                                # in "bbyus-trn-amlo-yieldoptimz-p01"
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", local_files_only=True)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
model.name_or_path

'openai/clip-vit-base-patch32'

# run model 

In [1]:
# 2025-11-17 Jupyter’s os.environ[...] = ... affects only the notebook kernel, not the subprocesses.
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/cube/"

import sys
sys.path.insert(0, "/home/jupyter/mluser/git/cube")

os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/cube/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] =  "max_split_size_mb:64,expandable_segments:True"

In [ ]:
ACCELERATE_CONFIG_FILE="/home/jupyter/mluser/git/cube/config/accelerate_config.yaml"

In [8]:
# !which accelerate
# test if it can be loaded correctly
from training.trainer import CubeDiffTrainer 

In [9]:
import huggingface_hub
print(huggingface_hub.constants.HF_HUB_CACHE)


/home/jupyter/.cache/huggingface/hub


In [10]:
# 2025-11-17 this snapshot was currupted due to internet/downloading broken
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/c237dc49a33fc61debc9276459120b7eac67e7ef


ls: cannot access '/home/jupyter/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/c237dc49a33fc61debc9276459120b7eac67e7ef': No such file or directory


In [11]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/

total 4.0K
drwxr-xr-x 2 jupyter jupyter 4.0K Nov 18 20:55 3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268


In [12]:
!ls -lh ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/3d74acf9a28c67741b2f4f2ea7635f0aaf6f0268

total 582M
-rw-r--r-- 1 jupyter jupyter 1.2K Nov 18 20:55 config.json
-rw-r--r-- 1 jupyter jupyter 513K Nov 18 20:55 merges.txt
-rw-r--r-- 1 jupyter jupyter 578M Nov 18 20:55 model.safetensors
-rw-r--r-- 1 jupyter jupyter  504 Nov 18 20:55 preprocessor_config.json
-rw-r--r-- 1 jupyter jupyter  588 Nov 18 20:55 special_tokens_map.json
-rw-r--r-- 1 jupyter jupyter 3.5M Nov 18 20:55 tokenizer.json
-rw-r--r-- 1 jupyter jupyter  774 Nov 18 20:55 tokenizer_config.json
-rw-r--r-- 1 jupyter jupyter 843K Nov 18 20:55 vocab.json


In [10]:
# 2025-11-17 removed the borken snapshot
# !rm -rf ~/.cache/huggingface/hub/models--openai--clip-vit-base-patch32/snapshots/c237dc49a33fc61debc9276459120b7eac67e7ef


In [13]:
from transformers import CLIPModel, CLIPProcessor

# This should now work without network calls
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True, local_files_only=True)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True, local_files_only=True, use_fast=False) # -> worked


In [2]:
!pip list | grep "diff\|hug\|acc\|trans\|torch\|peft\|tok\|xformers"

accelerate                               0.22.0
asttokens                                3.0.0
diffusers                                0.25.1
huggingface-hub                          0.25.1
sentence-transformers                    5.1.2
tokenizers                               0.15.2
torch                                    2.1.2
torch-model-archiver                     0.12.0
torchserve                               0.12.0
torchvision                              0.16.2
transformers                             4.35.2
xformers                                 0.0.23.post1


In [3]:
!accelerate launch --config_file $ACCELERATE_CONFIG_FILE train_cubediff.py --cfg ../../../config/tiny_fullrank.yaml > cubediff_log.txt

Loading pipeline components...:  43%|█████▌       | 3/7 [00:00<00:00, 11.21it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|█████████████| 7/7 [00:02<00:00,  2.62it/s]
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `prediction_type